In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [2]:
filename = 'QuanSongCi.txt'


# Read the data into a list of strings.
def read_data(filename):
    
    with open(filename, encoding = "utf-8") as f:
        data = f.read()
    return data

vocabulary = list(read_data(filename))
print('Data size', len(set(vocabulary)))

Data size 6010


In [3]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 5000


def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    index = dictionary.get(word, 0)
    if index == 0:  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

# Filling 4 global variables:
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0


Most common words (+UNK) [['UNK', 1196], ('。', 149620), ('\n', 117070), ('，', 108451), ('、', 19612)]
Sample data [1503, 1828, 2, 2, 40, 613, 47, 9, 111, 117] ['潘', '阆', '\n', '\n', '酒', '泉', '子', '（', '十', '之']


In [4]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builtin
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    context_words = [w for w in range(span) if w != skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]
    if data_index == len(data):
      buffer.extend(data[0:span])
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels


batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
        reverse_dictionary[labels[i, 0]])

1828 阆 -> 2 

1828 阆 -> 1503 潘
2 
 -> 2 

2 
 -> 1828 阆
2 
 -> 40 酒
2 
 -> 2 

40 酒 -> 2 

40 酒 -> 613 泉


In [5]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.
num_sampled = 64      # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)


graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  # Explanation of the meaning of NCE loss:
  #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  202.574157715
Nearest to 里: 胭, 箔, 拾, 邛, 搢, 总, 汪, 茅,
Nearest to 好: 垆, 声, 轧, 窬, 朅, 萼, 荃, 虬,
Nearest to 雨: 鞋, 尊, 私, 褶, 淰, 换, 辅, 昴,
Nearest to 归: 翦, 惭, 滓, 婢, 自, 猩, 涵, 嗔,
Nearest to 流: 队, 循, 礴, 扃, 疵, 侧, 龄, 笑,
Nearest to 重: 癸, 藉, 关, 简, 间, 守, 遑, 蕉,
Nearest to 梦: 庖, 缵, 肌, 滔, 坛, 绂, 屈, 磅,
Nearest to 知: 淮, 筋, 房, 候, 去, 享, 彳, 弈,
Nearest to 来: 六, 剩, 沃, 洎, 禅, 试, 厌, 愤,
Nearest to 去: 姨, 耆, 涸, 榕, 负, 眄, 巴, 逢,
Nearest to 声: 宛, 颃, 好, 锁, 银, 北, 孽, 狠,
Nearest to 梅: 治, 璀, 罚, 缕, 疆, 绉, 纵, 徙,
Nearest to 红: 浓, 摛, 躬, 卧, 矫, 濮, 膂, 符,
Nearest to 更: 恋, 岳, 财, 杯, 邴, 灼, 畏, 矫,
Nearest to 花: 字, 狸, 柴, 哗, 谱, 胭, 诉, 鲂,
Nearest to 自: 嗜, 归, 纽, 缩, 觌, }, 浙, 伽,
Average loss at step  2000 :  21.4972542681
Average loss at step  4000 :  5.30434288335
Average loss at step  6000 :  4.90093724012
Average loss at step  8000 :  4.68366949999
Average loss at step  10000 :  4.59345866412
Nearest to 里: 胭, 箔, 拾, 汪, 运, 老, 视,  ,
Nearest to 好: 垆, 萼, 声, 不,  , 纨, 轧, 踏,
Nearest to 雨: 鞋, 褶, 私, 鹣, 尊, 漠, 瘦, 阿,
Near

Average loss at step  94000 :  4.08339001381
Average loss at step  96000 :  4.0518479476
Average loss at step  98000 :  4.07891475117
Average loss at step  100000 :  4.05670115197
Nearest to 里: 中, 绕, 搢, 箔, 载, 逐, 醵, 视,
Nearest to 好: 囗, 宜, 佳, 坦, 缤, 乐, 菩, 何,
Nearest to 雨: 漠, 鹣, 昴, 雪, 霏, 黍, 数, 紧,
Nearest to 归: 去, 醒, 眠, 污, 早, 湍, 晚, 婢,
Nearest to 流: 囗, 队, 粮, 贳, 由, 飞, 湾, 呖,
Nearest to 重: 再, 慵, 关, 茧, 初, 珍, 同, 仓,
Nearest to 梦: 滔, 庖, 跻, 恰, 屈, 昨, 夺, 邵,
Nearest to 知: 信, 淮, 许, 关, 区, 料, 味, 筋,
Nearest to 来: 便, 教, 去, 到, 今, 乘, 彪, 醒,
Nearest to 去: 归, 逐, 浑, 来, 兵, 早, 趁, 住,
Nearest to 声: 闻, 锥, 锁, 蕾, 悲, 鸣, 宛, 笙,
Nearest to 梅: 杏, 治, 娄, 巧, 桂, 璀, 釂, 咒,
Nearest to 红: 酹, 坠, 蜡, 铎, 袜, 花, 蒲, 矫,
Nearest to 更: 霰, 便, 杯, 每, 煞, , 谁, 觇,
Nearest to 花: 锯, 柳, 穹, 碌, 褭, 红, 预, 芳,
Nearest to 自: 亏, 疑, 唯, 伽, 惟, 缱, 沌, 颜,


In [9]:
# Step 6: Visualize the embeddings.


# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.




try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt
  

  tsne = TSNE(
      perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]

except ImportError as ex:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')
  print(ex)


In [17]:
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')
    #plt.show()
    plt.savefig(filename)

from pylab import mpl
mpl.rcParams['axes.unicode_minus'] = False
plot_with_labels(low_dim_embs, labels, 'tsne.png')

In [18]:
np.save('embedding.npy', final_embeddings)

In [23]:
import json
with open('dictionary.json', 'w', encoding='utf-8') as fp:
  json.dump(dictionary, fp, ensure_ascii=False)
with open('reversed_dictionary.json', 'w', encoding='utf-8') as fp:
  json.dump(reverse_dictionary, fp, ensure_ascii=False)